In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip fake_source.zip
!unzip real_target.zip

In [ ]:
!pip install gradio_client

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor
from gradio_client import Client


OUTPUT_DIR = "1k_output"

client = Client("https://felixrosberg-face-swap.hf.space/", output_dir=OUTPUT_DIR)


def process_image(target_image, source_image):
    result = client.predict(
        target_image,
        source_image,
        0,
        0,
        "Compare",
        fn_index=1,
    )

In [ ]:
import time

start_time = time.time()
with ThreadPoolExecutor(max_workers=10) as executor:
    for target_image, source_image in zip(
        os.listdir("real_target"), os.listdir("fake_source")
    ):
        target_image = "real_target/" + target_image
        source_image = "fake_source/" + source_image
        executor.submit(process_image, target_image, source_image)
end_time = time.time()
print(f"Total run time: {end_time - start_time}")

In [ ]:
import os
import shutil

FINAL_OUTPUT_DIR = "face_swap_data"

os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)

# Loop through all subdirectories in the output directory
for subdir in os.listdir(OUTPUT_DIR):
    subdir_path = os.path.join(OUTPUT_DIR, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            if os.path.isfile(file_path):
                new_file_name = subdir + ".png"
                new_file_path = os.path.join(FINAL_OUTPUT_DIR, new_file_name)
                shutil.copy(file_path, new_file_path)

print("Total output files: ", len(os.listdir(FINAL_OUTPUT_DIR)))